In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%%time
train_df = pd.read_csv('train.csv')
artist_df = pd.read_csv('artists.csv')
profile_df = pd.read_csv('profiles.csv')

Wall time: 31.4 s


In [3]:
profile_df.set_index('user', inplace=True)

In [21]:
print profile_df.shape
profile_df.head()

(233286, 3)


,sex,age,country
user,,,
fa40b43298ba3f8aa52e8e8863faf2e2171e0b5d,f,25,Sweden
5909125332c108365a26ccf0ee62636eee08215c,m,29,Iceland
d1867cbda35e0d48e9a8390d9f5e079c9d99ea96,m,30,United States
63268cce0d68127729890c1691f62d5be5abd87c,m,21,Germany
02871cd952d607ba69b64e2e107773012c708113,m,24,Netherlands


In [22]:
for k in range(10):
    profile_df[str(10*k) + 's'] = ((profile_df.age >= 10*k) & (profile_df.age < 10*k+10)).astype(int)
print profile_df.shape

(233286, 13)


C:\Users\Madhu\Anaconda\lib\site-packages\IPython\kernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app


In [24]:
%%time
profile_df = pd.concat([profile_df, pd.get_dummies(profile_df.sex)], axis=1)
profile_df = pd.concat([profile_df, pd.get_dummies(profile_df.age)], axis=1)
profile_df = pd.concat([profile_df, pd.get_dummies(profile_df.country)], axis=1)

Wall time: 3.38 s


In [25]:
print profile_df.shape
profile_df.head()

,sex,age,country,0s,10s,20s,30s,40s,50s,60s,...,Vanuatu,Venezuela,Viet Nam,"Virgin Islands, British","Virgin Islands, U.s.",Wallis and Futuna,Western Sahara,Yemen,Zambia,Zimbabwe
user,,,,,,,,,,,,,,,,,,,,,
fa40b43298ba3f8aa52e8e8863faf2e2171e0b5d,f,25,Sweden,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5909125332c108365a26ccf0ee62636eee08215c,m,29,Iceland,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
d1867cbda35e0d48e9a8390d9f5e079c9d99ea96,m,30,United States,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
63268cce0d68127729890c1691f62d5be5abd87c,m,21,Germany,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
02871cd952d607ba69b64e2e107773012c708113,m,24,Netherlands,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
%%time
i = 0
for tup in artist_df.itertuples():
    if i < 500:
        profile_df[tup[1]] = 0
    i += 1
print profile_df.shape

(233286, 867)
Wall time: 5.12 s


In [32]:
print profile_df.shape
profile_df.head()

(233286, 867)


,sex,age,country,0s,10s,20s,30s,40s,50s,60s,...,a5f92d47-b562-4582-b267-71963a515f4c,ba147ea0-60c4-4a40-9da6-0d5c2b2e7b03,a4f54716-24d9-4c6b-8ba7-1d7e7a5173d0,1fc494a1-9109-4081-a455-2d05bea9d2bf,efc8a006-d0c6-4a9b-8cb1-91ca770fa2b9,c5f5dc27-3059-49c0-ae45-5009a01bb9ec,1c3919b2-43ca-4a4a-935d-9d50135ec0ef,4bdcee62-4902-4773-8cd1-e252e2e31225,a74b1b7f-71a5-4011-9441-d0b5e4122711,f5426431-f490-4678-ad44-a75c71097bb4
user,,,,,,,,,,,,,,,,,,,,,
fa40b43298ba3f8aa52e8e8863faf2e2171e0b5d,f,25,Sweden,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5909125332c108365a26ccf0ee62636eee08215c,m,29,Iceland,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
d1867cbda35e0d48e9a8390d9f5e079c9d99ea96,m,30,United States,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
63268cce0d68127729890c1691f62d5be5abd87c,m,21,Germany,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
02871cd952d607ba69b64e2e107773012c708113,m,24,Netherlands,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
%%time
for row in train_df.itertuples():
    if row[2] in profile_df:
        profile_df.loc[row[1], row[2]] = row[3]

Wall time: 47min 43s


In [ ]:
import cPickle as pickle
with open('profile_df.p', 'w') as p:
    pickle.dump(profile_df, p)

In [41]:
for row in train_df.head(10).itertuples():
    if row[2] in profile_df:
        print row

(0, 'eb1c57ddc9e0e2d005169d3a1a96e8dd95e3af03', '5a8e07d5-d932-4484-a7f7-e700793a9c94', 554)
(2, 'da9cf3f557161d54b76f24db64be9cc76db008e3', 'eeb1195b-f213-4ce1-b28c-8565211f8e43', 708)
(3, '8fa49ab25d425edcf05d44bfc1d5aea895287d81', 'a1419808-65d3-4d40-998c-1a0bac65eabc', 265)
(4, 'b85fcaef67d2669cd99b334b5e8c8705263db2cf', 'a3cb23fc-acd3-4ce0-8f36-1e5aa6a18432', 220)
